# Formula 1: Co2 Calculations

This notebook will calculate the amount of fuel used by F1 cars now and in the past to work out how much Co2 has been saved. Then we will calculate potential Co2 savings from optimizing the season race schedule to save as much Co2 as possible.

In [12]:
# Import Libraties
import pandas as pd
import matplotlib as plt
import requests
import geopy
from geopy .geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
import geopandas as gpd
import folium

#from ipyleaflet import Map, AntPath, MeasureControl
#import ipywidgets


 Input Data
 
 All sources provided, change sources and variables (below) to update all the calculations

- F1 Fuel website 
https://motorsportexplained.com/f1-fuel/
-  Co2 per litre of fuel https://www.carbonindependent.org/17.html#:~:text=The%20CO%202%20emissions%20from%20petrol%20and%20diesel,per%20gallon%20%28since%201%20gallon%20is%204.546%20litres%29.

In [5]:
# Variables
fuel_qty_2019 = 110
cars_per_race = 20
fuel_co2_litre = 2.4



In [6]:
# Max fuel used per race
total_race_fuel = fuel_qty_2019*cars_per_race
print(total_race_fuel)

2200


In [7]:
# Max Co2 per race (from fuel use)
co2_race2019 = total_race_fuel*fuel_co2_litre
print("During the 2019 season a single F1 race emits", co2_race2019, "kg of Co2.")

During the 2019 season a single F1 race emits 5280.0 kg of Co2.


# Race Schedule 2023

Webscrape wikipedia page to get the 2023 season schedule, then geocode location of each track and get lat lon.

In [8]:
page_url_season_2023 = 'https://en.wikipedia.org/wiki/2023_Formula_One_World_Championship'

html = requests.get(page_url_season_2023, timeout = 5, verify = True)

html.text[:1000]

dfs2023 = pd.read_html(html.text)

dfs2023[1]

df = dfs2023[1]

df['Town']= df['Circuit'].str.rsplit(',').str[-1] 

# Nominate geocoder for OpenStreetMap data with RateLimiter
geocoder = RateLimiter(Nominatim(user_agent='tutorial').geocode, min_delay_seconds=1)
df['Location']= df['Town'].apply(geocoder)

# add latitude and longitude to dataframe
df['Latitude']=df['Location'].apply(lambda loc: loc.latitude if loc else None)
df['Longitude']=df['Location'].apply(lambda loc: loc.longitude if loc else None)

df.to_csv('season2023.csv', index = True)

df



,Round,Grand Prix,Circuit,Race date,Town,Location,Latitude,Longitude
0,1,Bahrain Grand Prix,"Bahrain International Circuit, Sakhir",5 March,Sakhir,"(ساخر, شهيدې حساس ولسوالۍ, روزگان ولايت, افغان...",32.953941,65.540016
1,2,Saudi Arabian Grand Prix,"Jeddah Corniche Circuit, Jeddah",19 March,Jeddah,"(جدة, محافظة جدة, منطقة مكة المكرمة, 23434, ال...",21.581009,39.165361
2,3,Australian Grand Prix,"Albert Park Circuit, Melbourne",2 April,Melbourne,"(Melbourne, City of Melbourne, Victoria, Austr...",-37.814245,144.963173
3,4,Azerbaijan Grand Prix,"Baku City Circuit, Baku",30 April,Baku,"(Bakı, Yasamal rayonu, Bakı İnzibati Ərazisi, ...",40.375588,49.832801
4,5,Miami Grand Prix,"Miami International Autodrome, Miami Gardens, ...",7 May,Florida,"(Florida, United States, (27.7567667, -81.4639...",27.756767,-81.463983
5,6,Monaco Grand Prix,"Circuit de Monaco, Monaco",28 May,Monaco,"(Monaco, (43.7323492, 7.4276832))",43.732349,7.427683
6,7,Spanish Grand Prix,"Circuit de Barcelona-Catalunya, Montmeló",4 June,Montmeló,"(Montmeló, Vallès Oriental, Barcelona, Catalun...",41.551519,2.248081
7,8,Canadian Grand Prix,"Circuit Gilles Villeneuve, Montreal",18 June,Montreal,"(Montréal, Agglomération de Montréal, Montréal...",45.503182,-73.569806
8,9,Austrian Grand Prix,"Red Bull Ring, Spielberg",2 July,Spielberg,"(Spielberg, Gemeinde Melk, Bezirk Melk, Nieder...",48.227452,15.357276
9,10,British Grand Prix,"Silverstone Circuit, Silverstone",9 July,Silverstone,"(Silverstone, West Northamptonshire, England, ...",52.087729,-1.024118


In [19]:
#   Create a new geodataframe for race tracks
race_Tracks = gpd.GeoDataFrame(df[['Round', 'Grand Prix', 'Town', 'Race date', ]],
                             geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']),
                             crs='epsg:4326')
race_Tracks

,Round,Grand Prix,Town,Race date,geometry
0,1,Bahrain Grand Prix,Sakhir,5 March,POINT (65.54002 32.95394)
1,2,Saudi Arabian Grand Prix,Jeddah,19 March,POINT (39.16536 21.58101)
2,3,Australian Grand Prix,Melbourne,2 April,POINT (144.96317 -37.81425)
3,4,Azerbaijan Grand Prix,Baku,30 April,POINT (49.83280 40.37559)
4,5,Miami Grand Prix,Florida,7 May,POINT (-81.46398 27.75677)
5,6,Monaco Grand Prix,Monaco,28 May,POINT (7.42768 43.73235)
6,7,Spanish Grand Prix,Montmeló,4 June,POINT (2.24808 41.55152)
7,8,Canadian Grand Prix,Montreal,18 June,POINT (-73.56981 45.50318)
8,9,Austrian Grand Prix,Spielberg,2 July,POINT (15.35728 48.22745)
9,10,British Grand Prix,Silverstone,9 July,POINT (-1.02412 52.08773)


In [23]:
m = folium.Map()

for index, row in df.iterrows():
    folium.Marker([row['Longitude.lng'], row['Latitude.lat']], popup=row['Grand Prix']).add_to(f_map)

m

KeyError: 'Longitude.lng'